In [ ]:
# default_exp pipeline

# Module pipeline

> Here we have the Nodes wrapping around our core functions, which can be added to a Nipype workflow to process your input images. These definitions here are our custom functions and Nipype Nodes knotted together, which creates a working pipeline.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
%%capture
#export
import os, sys
from shutil import which

import pipetography.core as ppt

from nipype import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.interfaces.mrtrix3.utils import BrainMask, TensorMetrics, DWIExtract, MRMath
from nipype.interfaces.mrtrix3.preprocess import MRDeGibbs, DWIBiasCorrect
from nipype.interfaces.mrtrix3.reconst import FitTensor
from nipype.interfaces import ants
from nipype.interfaces import fsl

#### A `pipeline` class that puts everything together.

In [ ]:
# export
class pipeline:
    """
    A class containing functionalities and inputs to the image processing pipeline.
    For step-by-step preprocessing example instead of a connected pipeline, see `core` page.
    
    This pipeline makes the following assumptions:
        - Data input is in valid BIDS format
        - Align all anat and dwi images to MNI space via ACPC HCP procedure
    
    Input: 
        BIDS_dir (str): Valid BIDS directory with DWI modality
        RPE_design (str): Reverse phase encoding design for DWI
        Regrid (Bool): [False] Whether to resample DWI to a template image's voxel grid
        mrtrix_nthreads (Int): [0] Number of paralel threads to perform mrtrix tasks, default to 0, which disables multi-threading
    Attributes:
        data_dir (str): input BIDS data directory
        sub_list (List): List of subject IDs read from the BIDS directory
        layout (PyBIDS Layout): PyBIDS Layout for querying
        bfiles_fsl (tuple)
    """

    def __init__(self, BIDS_dir="data", RPE_design="rpe_none", Regrid=False, mrtrix_nthreads=0):
        # Nodes are initialized with the bare minimum of inputs or default parameters
        # Remaining inputs are either connected in the Nipype workflow or are user inputs
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # Input variables to pipeline
        self.bids_dir = BIDS_dir
        self.RPE_design = RPE_design
        self.Regrid = Regrid
        self.mrtrix_nthreads = mrtrix_nthreads
        
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # Generate BIDS Layout, and create subject ID list:
        self.sub_list, self.layout = ppt.get_subs(self.data_dir)
        
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # Create BIDS data input node:
        self.subject_source = Node(IdentityInterface(fields=["subject_id"]), name="sub_source")
        # This node will create iterables from the subject ID list:
        self.subject_source.iterables = [("subject_id", self.sub_list)]
        # string templates for images:
        self.anat_file = os.path.join(
            'sub-{subject_id}', 'ses-*', 'anat', 'sub-{subject_id}_ses-*_T1w.nii.gz'
        )
        self.dwi_file = os.path.join(
            "sub-{subject_id}", "ses-*", "dwi", "sub-{subject_id}_ses-*_dwi.nii.gz",
        )
        self.b_files = os.path.join(
            "sub-{subject_id}", "ses-*", "dwi", "sub-{subject_id}_ses-*_dwi.bv*"
        )
        self.sub_template = {"anat": self.anat_file, "dwi": self.dwi_file, "b_files": self.b_files}        
       
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # node for selecting images and bfiles
        self.select_files = Node(
            SelectFiles(self.sub_template, base_directory=self.bids_dir), name="select_files",
        )
        self.sub_grad_files = MapNode(
            Function(input_names=["sub_dwi"], output_names=["fslgrad"], function=ppt.ppt.get_sub_gradfiles),
            name = 'sub_grad_files',
            iterfield = "sub_dwi"
        )
        self.bfiles_input = Node(
            Function(
                input_names=["in_List"], output_names=["out_path"], function=ppt.get_bfiles_tuple,
            ),
            name="select_bfiles",
        )
        
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # ACPC HCP MNI152 alignment nodes:
        # first define template location in FSL:
        self.MNI_template = os.path.expandvars('$FSLDIR/data/standard/MNI152_T1_1mm.nii.gz')
        # The following nodes creates transformation matrices and applys them to align our T1 image
        self.reduceFOV = MapNode(fsl.utils.RobustFOV(), name="reduce_FOV", iterfield="in_file")
        self.xfminverse = MapNode(fsl.utils.ConvertXFM(), name="transform_inverse", iterfield="in_file")
        self.flirt = MapNode(fsl.preprocess.FLIRT(), name="FLIRT", iterfield="in_file")
        self.concatxfm = MapNode(fsl.utils.ConvertXFM(), name="concat_transform", iterfield=["in_file", "in_file2"])
        self.alignxfm = MapNode(ppt.fslaff2rigid(), name='aff2rigid', iterfield="in_file")
        self.ACPC_warp = MapNode(fsl.preprocess.ApplyWarp(), name='apply_warp', iterfield=["in_file", "premat"])
        
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # FreeSurfer recon-all mainly for WM segmentation
        # we are greedy so we just let it segment and produce everything
        self.reconall = MapNode(ReconAll(), name='FSrecon', iterfield="T1_files")
        
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # Preparation nodes (mask, check gradient, convert):
        # first convert our input nifti to mrtrix3 format
        self.mrconvert = MapNode(ppt.Convert(), name='mrtrix_image', iterfield=["in_file", "grad_fsl")
        # create raw DWI brain mask
        self.createMask = MapNode(BrainMask(), name='dwi2mask', iterfield='in_file')
        # check for gradients and create a new image with correct gradients
        self.GradCheck = MapNode(ppt.GradCheck(), name='dwigradcheck', iterfield='in_file')
        self.NewGradMR = MapNode(ppt.Convert(), name='mrconvert', iterfield = "in_file")
        
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # Preprocessing nodes:
        # First MPPCA denoise with dwidenoise
        self.denoise = MapNode(ppt.dwidenoise(), name="denoise", iterfield="in_file")
        # Gibb's ringing artifact removal:
        self.degibbs = MapNode(MRDeGibbs(), name="ringing_removal", iterfield="in_file")
        # DWI Distortion correction with mrtrix3's dwifslpreproc:
        self.fslpreproc = MapNode(ppt.dwipreproc(), name = "dwifslpreproc", iterfield="in_file")
        # Bias field correction using ANTS:
        self.biascorrect = MapNode(ppt.BiasCorrect(), name = 'dwibiascorret', iterfield="in_file")
        # Rician background noise removal requires multiple nodes:
        # create new gradient file because we have changed the data extensively now:
        self.grad_info = MapNode(ppt.MrInfo(), name = 'New Gradient', iterfield = "in_file")
        # get background low noise map:
        self.low_noise_map = MapNode(ppt.CheckNIZ(), name = 'Low Noise Map', iterfield="isfininte")
        # Compute Rician noise:
        self.rician_noise = MapNode(ppt.RicianNoise(), name = 'Rician Noise', iterfield = "in_file")
        self.check_rician = MapNode(ppt.CheckNIZ(), name = 'Noise Comparison', iterfield = "isfinite")
        self.convert_rician = MapNode(ppt.Convert(), name = "Convnert Rician", iterfield = "in_file")
        # DWI image intensity normalization:
        # first create a DWI brain mask:
        self.dwi_mask = MapNode(BrainMask(), name='dwi2mask', iterfield='in_file')
        # create a FA WM mask:
        self.fit_tensor = MapNode(FitTensor(), name='dwi2tensor', iterfield='in_file')
        self.tensor_FA = MapNode(TensorMetrics(), name='tensor2metrics', iterfield='in_file')
        self.wm_mask = NapNode(ppt.MRThreshold(), name = 'mrthreshold', iterfield='in_file')
        # finally intensity normalization:
        self.norm_intensity = MapNode(ppt.DWINormalize(), name='dwinormalise', iterfield='in_file')
        
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # Create new B0 and DWI brain masks with updated images to prep for re-alignement to MNI space
        self.sub_b0extract = MapNode(DWIExtract(),  name='b0extract', iterfield='in_file')
        self.sub_b0mean = MapNode(MRMath(), name='mrmath_mean', iterfield='in_file')
        self.sub_b0mask = MapNode(BrainMask(), name='dwi2mask', iterfield='in_file')
        # convert from mif to nift
        self.sub_convert_dwi = MapNode(ppt.Convert(), name="dwi to nii", iterfield="in_file")
        self.sub_convert_mask = MapNode(ppt.Convert(), name="mask to nii", iterfield="in_file")
        # apply mask to B0 volume:
        self.sub_apply_mask = MapNode(fsl.ApplyMask(), name='Apply Mask', iterfield="in_file")
        
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # Align DWI to ANAT w/ ACPC HCP MNI152 space
        # first extract the brain:
        self.t1_bet = MapNode(fsl.preprocess.BET(), name='fsl_bet', iterfield="in_file")
        # align DWI to ANAT... get transformation matrix first
        self.epi_reg = MapNode(fsl.epi.EpiReg(), name='fsl_epireg', iterfield='epi')
        # convert transformation matrix to mrtrix format
        self.acpc_xfm = MapNode(ppt.TransConvert(), name='transformconvert', iterfield="flirt_xfm")
        # apply the transformation:
        self.apply_xfm = MapNode(ppt.MRTransform(), name='mrtransform', iterfield="in_file")
        
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # if regrid = True, do regridding, if False, we skip these nodes in workflow:
        # this is mainly done to isotropically upsample your DWI data
        # Partial upsampling is not supported and not recommended
        self.regrid = MapNode(ppt.MRRegrid(), nam='mrgrid', iterfield='in_file')
        
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # Recreate final DWI/Reference images and their corresponding gradient/info files:
        self.mni_b0extract = MapNode(DWIExtract(), name='b0extract', iterfield='in_file')
        self.mni_b0mean = MapNode(MRMath(), name='mrmath_mean', iterfield='in_file')
        self.mni_b0mask = MapNode(BrainMask(), name='dwi2mask', iterfield='in_file')
        self.mni_convert_dwi = MapNode(ppt.Convert(), name='dwi to nii', iterfield='in_file')
        self.mni_convert_mask  = MapNode(ppt.Convert(), name='mask to  nii', iterfield='in_file')
        self.mni_apply_mask = MapNode(fsl.ApplyMask(), name='Apply Mask', iterfield='in_file')
        self.mni_dwi = MapNode(ppt.Convert(), name='MNI space outputs', iterfield='in_file')
        
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # Lastly, registration of atlases, these atlases will define ROIs for connectome generation:
        # These steps should be moved to post-processing pipeline
#         self.atlas_dir = None
#         self.atlas_names = None
#         self.atlas_source = None
#         self.select_atlas = None
#         # b0 volume brain and mask extraction:
#         self.b0extract = MapNode(DWIExtract(), name="dwiextract", iterfield="in_file")
#         self.b0mean = MapNode(MRMath(), name="mrmath", iterfield="in_file")
#         self.fsl_bet = MapNode(fsl.BET(), name="brain_extraction", iterfield="in_file")
#         # ants coregistration:
#         self.linear_coreg = MapNode(
#             ants.Registration(), name="linear_registration", iterfield="fixed_image",
#         )
#         self.nonlinear_coreg = MapNode(
#             ants.Registration(), name="nonlinear_registration", iterfield="fixed_image",
#         )
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # data sink is the output directory of the entire pipeline:
        self.datasink = Node(DataSink(base_directory=os.path.join(self.bids_dir, 'derivatives')), name="datasink")
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # workflow for nipype, this will be defined after connecting the working nodes
        self.workflow = None
    
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #    
    def check_environment(self):
        """
        Check your computing environment for FSL environment variables `FSLOUTPUTTYPE` and `FSLDIR`
        Check if ANTs PATH is included in your environment
        Check if mrtrix3 is in your PATH
        Check your freesurfer envrionment variable
        Looks for matlab compiler run time in freesurfer directory: https://surfer.nmr.mgh.harvard.edu/fswiki/MatlabRuntime
        """
        assert "FSLOUTPUTTYPE" in os.environ
        assert "FSLDIR" in os.environ
        if "FSLOUTPUTTYPE" in os.environ:
            print("FSLOUTPUTTYPE is valid")
        else:
            sys.exit("FSLOUTPUTTYPE is not defined, make sure FSL is configured!")

        if "FSLDIR" in os.environ:
            print("FSLDIR is valid")
        else:
            sys.exit("FSLOUTPUTTYPE is not defined, make sure FSL is configured!")
        
        #ANTS:
        assert "ANTSPATH" in os.environ
        print("ANTS is valid")
        #mrtrix:
        assert which('mrview') is not None
        print("mrtrix3 is valid")
        #freesurfer:
        assert "FREESURFER_HOME" in os.environ
        print("Freesurfer path found")
        #matlab compiler run time
        assert os.path.isdir(os.path.expandvars('$FREESURFER_HOME/MCRv80'))
        
        return None
    
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #    
#     def set_datasink(self):
#         """
#         Set directory where output files will be placed
#         """
#         output_dir = str(input('Please indicate an output directory: '))
#         self.datasink.inputs.base_directory = output_dir
    
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
    # List of functions for setting node inputs:
    def reduceFOV_inputs(
        self,
        xfm_mat="roi2full.mat",
        out_vol="robustfov.nii.gz",
    ):
        """
        Inputs to FSL's robustfov command: robustfov -i [input] -m [matrix name] -r [volume name]
        """
        self.reduceFOV.inputs.out_transform = xfm_mat
        self.reduceFOV.inputs.out_roi = out_vol
    
    
    def flirt_inputs(
        self,
        template=self.MNI_template,
        method='spline',
        mat_file='roi2std.mat',
        out_file='acpc_mni.nii.gz'
    ):
        """
        Inputs to FSL's FLIRT:
        -Iterable input is robustfov.nii.gz (robustFOV's output)
        """
        self.FLIRT.inputs.reference = template
        self.FLIRT.inputs.interp = method
        self.FLIRT.inputs.out_matrix_file = mat_file
        self.FLIRT.inputs.out_file = out_file
    
    
    def xfminverse_inputs(
        self,
        inverse=True,
        out_file='full2roi.mat'
    ):
        """
        Inputs to FSL's convert_xfm command: convert_xfm -omat [filename] -inverse [input]
        """
        self.xfminverse.inputs.out_file = out_file
        self.xfminverse.inputs.invert_xfm = inverse
    
    
    def concatxfm_inputs(
        self,
        concat = True,
        omat = 'full2std.mat'
    ):
        """
        Inputs for transformation matrix concatenation node
        The matrices being concatenated are 1) flirt's output mat file  (in_file2) and 2) inverse mat file from the first convert_xfm (in_file)
        """
        self.concatxfm.inputs.concat_xfm=True
        self.concatxfm.inputs.out_file=omat
        
    
    def aff2rigid_inputs(
        self,
        out_file='outputmatrix'
    ):
        self.alignxfm.inputs.out_file=out_file
    
    
    def acpcwarp_inputs(
        self,
        out_file='acpc_t1.nii.gz',
        rel_warp = True,
        method = 'spline',
        template = self.MNI_template,
    ):
        """
        Inputs for FSL's `applywarp`
        We want spline interpolation, relative warp field, premat = aff2rigid mat output.
        """
        self.ACPC_warp.inputs.out_file = out_file
        self.ACPC_warp.inputs.relwarp = rel_warp
        self.ACPC_warp.inputs.interp = method
        self.ACPC_warp.inputs.ref_file = template
        
    
    # Freesurfer recon-all
    def recon_inputs(
        self,
        hipp = True,
    ):
        """
        We will multiproc the entire workflow, so parallel here is not recommended.
        Only recommend parallel for single subject runs or group level runs.
        """
        self.reconall.inputs.directive='all' # we always want recon-all -all
        self.reconall.inputs.subjects_dir = os.path.join(self.BIDS_dir,'derivatives','freesurfer')
        self.reconall.inputs.hippocampal_subfields_T1 = True # we want this for compeleteness of data
        #self.reconall.inputs.subject_id
        
    
    # Convert to mrtrix3 file format:
    def mrconvert_inputs(
        self,
        out_file = 'raw_dwi.mif',
        export_mrtrix3_grad = True,
        out_bfile = 'raw_dwi.b',
    ):
        """
        Convert a DWI to mrtrix3's image format and gradient files
        Workflow connected inputs not listed here:
            - grad_fsl - gradient files in fsl format
            - in_file - raw dwi that you want to convert to mrtrix3 format
        """
        self.mrconvert.inputs.out_file = out_file
        self.mrconvert.inputs.export_grad = export_mrtrix3_grad
        self.mrconvert.inputs.out_bfile = out_bfile
        self.mrconvert.inputs.force = True
        self.mrconvert.inputs.quiet = True
        self.mrconvert.iputs.nthreads = self.mrtrix_nthreads
    
    
    # DWI Preprocessing mask creation:
    def createMask_inputs(
        self,
        out_file = 'b0_brain_mask.mif'
    ):
        """
        Create brain mask from B0 DWI volumes for preprocessing tasks.
        in_file will be connected in workflow
        """
        self.createMask.inputs.out_file = out_file
        self.createMask.inputs.nthreads = self.mrtrix_nthreads
        
    
    # gradient check and create files with corrected gradient:
    def GradCheck_inputs(
        self,
        out_bfile = 'corrected.b',
        export_mrtrix3_grad = True,
    ):
        """
        Set inputs to dwigradcheck
        Workflow inputs:
            in_file = input from mrconvert Node 
            mask_file = input from createMask Node
            grad_file = input from mrconvert Node
        """
        self.GradCheck.inputs.export_grad = export_mrtrix3_grad
        self.GradCheck.inputs.out_bfile = out_bfile
        self.GradCheck.inputs.force = True
        self.GradCheck.inputs.quiet = True
        self.GradCheck.inputs.nthreads = self.mrtrix_nthreads
    
    
    # Create new dwi image with corrected gradients:
    def NewGradMR_inputs(
        self,
        out_file = 'cdwi.mif'
    ):
        """
        Inputs for creating gradient corrected MR
        Outputs a correced dwi file
        Workflow connected inputs:
            in_file - output from mrconvert Node
            grad_file - output from GradCheck Node (out_bfile)
        """
        self.NewGradMR.inputs.out_file = out_file
        self.NewGradMR.inputs.force = True
        self.NewGradMR.inputs.quiet = True
        self.NewGradMR.inputs.nthreads=self.mrtrix_nthreads
        
    
    # MPPCA Denoise with dwidenoise from mrtrix3
    def denoise_inputs(
        self,
        out_file="denoised.mif",
        noise_file="noise_map.mif",
        nthreads=None,
    ):
        """
        Set inputs to denoise node
        """
        self.denoise.inputs.out_file = out_file
        self.denoise.inputs.noise = noise_file
        self.denoise.inputs.force = True
        self.denoise.inputs.quiet = True
        self.denoise.inputs.nthreads = self.mrtrix_nthreads

            
    # Gibbs ringing artifact removal
    def DeGibbs_inputs(self, out_file="ringing_removed.nii.gz"):
        """
        Set inputs to ringing removal node
        """
        self.DeGibbs.inputs.out_file = out_file
        self.DeGibbs.inputs.force=True
        self.DeGibbs.inputs.quiet=True
        self.DeGibbs.inputs.nthreads = self.mrtrix_nthreads

    
    # dwifslpreproc - Distortion correction with mrtrix3
    def fslpreproc_inputs(
        self,
        pe_dir,
        eddy_options = '"--slm=linear "',
        readout = 
        out_file="preproc.nii.gz",
    ):
        """
        Set inputs to mrtrix3's dwipreproc.
        Arguments:
            out_file: name of output file
            rpe_options (str): specify phase encoding design -rpe_+ 'none/pair/all/header'
            pe_dir (str): phase encoding direction(i,j,k)/(0,1,2)/(RL,PA,IS), can be extracted from JSON & BIDS Layout
            eddy_options (str): '"--slm=linear "' aditional commandline options to eddy command, with at least 1  space
            RO_time = '-readout_time' TotalReadoutTime from BIDS json. Defaults to 0.1 if not provided
        """
        self.fslpreproc.inputs.out_file = out_file
        self.fslpreproc.inputs.pe_dir = pe_dir
        self.fslpreproc.inputs.eddy_options = eddy_options
        #if self.RPE_design == 'rpe_none':
            #self.fslpreproc.inputs.rpe_options = self.RPE_design # -rpe_none
        #elif self.RPE_design == 'rpe_pair':
        self.fslpreproc.inputs.rpe_options = self.RPE_design
        # inputs that can be read from BIDS Layout:
        # TotalReadoutTime
        try:
            self.mrt_preproc.inputs.RO_time = self.layout.get_metadata()['TotalReadoutTime']
        except:
            print('No TotalReadoutTime in BIDS DWI JSON file, setting to 0.1')
        else:
            self.mrt_preproc.inputs.RO_time = 0.1
        
        # RPE_DIR
        try:
            self.fsl_preproc.inputs.pe_dir = self.layout.get_metadata()['PhaseEncodingDirection']
        except:
            print('No Phase Encoding direction in BIDS JSON metadata')
    
    
    def ants_bfc_inputs(self, out_file="biasfieldcorrected.nii.gz", dims=4):
        """
        Set inputs to ANTs bias field correction node, currently incomplete
        """
        self.ants_bfc.inputs.out_file = out_file
        self.ants_bfc.inputs.dims = dims

    

    def atlas_inputs(self, atlas_dir, atlas_names):
        """
        Input where to load atlases for ROIs and co-registration
        """
        self.atlas_dir = atlas_dir
        self.atlas_names = atlas_names
        atlas_template = {"atlas": atlas_dir + "{file_name}"}
        self.atlas_source = Node(IdentityInterface(fields=["atlas_name"]), name="atlas_source")
        self.atlas_source.iterables = [("atlas_name"), self.atlas_names]
        self.select_atlas = Node(SelectFiles(atlas_template), name="select_atlases")
        self.select_atlas.base_directory = self.atlas_dir

    def b0extract_inputs(self, out_file="dwi_b0.nii.gz", bzero=True):
        self.b0extract.inputs.out_file = out_file
        self.b0extract.inputs.bzero = bzero

    def b0mean_inputs(self, out_file="dwi_b0_mean.nii.gz", axis=3):
        self.b0mean.inputs.operation = "mean"
        self.b0mean.inputs.axis = axis
        self.b0mean.inputs.out_file = out_file

    def bet_inputs(
        self, robust=True, mask=True, frac=0.25, out_file="dwi_brain.nii.gz"
    ):
        if robust is True:
            self.fsl_bet.inputs.robust = robust
        else:
            self.fsl_bet.inputs.reduce_bias = False

        self.fsl_bet.inputs.mask = mask
        self.fsl_bet.inputs.frac = 0.25
        self.fsl_bet.inputs.out_file = out_file
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
#     def linear_coreg_inputs(
#         self,
#         dims=3,
#         prefix="atlas_in_dwi_affine",
#         collapse=True,
#         transform=["Affine"],
#         parameters=[(0.1,)],
#         metric=["MI"],
#         weight=[1],
#         radius_bins=[64],
#         num_iter=[[500, 200, 200, 100]],
#         threshold=[1e-06],
#         window_size=[10],
#         sigmas=[[4, 2, 1, 0]],
#         units=["vox"],
#         shrink=[[8, 4, 2, 1]],
#         histogram=True,
#         output="atlas_in_dwi_affine.nii.gz",
#     ):
#         self.linear_coreg.inputs.dimension = dims
#         self.linear_coreg.inputs.output_transform_prefix = prefix
#         self.linear_coreg.inputs.collapse_output_transforms = collapse
#         self.linear_coreg.inputs.transforms = transform
#         self.linear_coreg.inputs.transform_parameters = parameters
#         self.linear_coreg.inputs.metric = metric
#         self.linear_coreg.inputs.metric_weight = weight  # default, value ignored by ANTS
#         self.linear_coreg.inputs.radius_or_number_of_bins = radius_bins
#         # -convergence
#         self.linear_coreg.inputs.number_of_iterations = num_iter
#         self.linear_coreg.inputs.convergence_threshold = threshold
#         self.linear_coreg.inputs.convergence_window_size = window_size
#         # -s
#         self.linear_coreg.inputs.smoothing_sigmas = sigmas
#         self.linear_coreg.inputs.sigma_units = units
#         # -f
#         self.linear_coreg.inputs.shrink_factors = shrink
#         self.linear_coreg.inputs.use_histogram_matching = histogram  # -u flag
#         self.linear_coreg.inputs.output_warped_image = output
    
#     def nonlinear_coreg_inputs(
#         self,
#         dims=3,
#         prefix="atlas_in_dwi_syn",
#         collapse=True,
#         transform=['SyN'],
#         parameters=[(0.1,)],
#         metric=["MI"],
#         weight=[1],
#         radius_bins=[64],
#         num_iter=[[500, 200, 200, 100]],
#         threshold=[1e-06],
#         window_size=[10],
#         sigmas=[[4, 2, 1, 0]],
#         units=["vox"],
#         shrink=[[8, 4, 2, 1]],
#         histogram=True,
#         output="atlas_in_dwi_syn.nii.gz",
#     ):
#         self.nonlinear_coreg.inputs.dimension = dims
#         self.nonlinear_coreg.inputs.output_transform_prefix = prefix
#         self.nonlinear_coreg.inputs.collapse_output_transforms = collapse
#         self.nonlinear_coreg.inputs.transforms = transform
#         self.nonlinear_coreg.inputs.transform_parameters = parameters
#         self.nonlinear_coreg.inputs.metric = metric
#         self.nonlinear_coreg.inputs.metric_weight = weight  # default, value ignored by ANTS
#         self.nonlinear_coreg.inputs.radius_or_number_of_bins = radius_bins
#         # -convergence
#         self.nonlinear_coreg.inputs.number_of_iterations = num_iter
#         self.nonlinear_coreg.inputs.convergence_threshold = threshold
#         self.nonlinear_coreg.inputs.convergence_window_size = window_size
#         # -s
#         self.nonlinear_coreg.inputs.smoothing_sigmas = sigmas
#         self.nonlinear_coreg.inputs.sigma_units = units
#         # -f
#         self.nonlinear_coreg.inputs.shrink_factors = shrink
#         self.nonlinear_coreg.inputs.use_histogram_matching = histogram  # -u flag
#         self.nonlinear_coreg.inputs.output_warped_image = output
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
    # Connect the nodes we defined above into a workflow:
    def connect_nodes(self, wf_name="pipetography"):
        """
        Connect nodes one by one and create Nipype workflow
        Input: wfname (str): name your workflow, default is pipetography
        """
        # a workflow that will be filled in with connections:
        self.workflow = Workflow(name=wf_name, base_dir=os.path.join(self.data_dir, "derivatives"))
        # connections:
        # ++++++++++++++++++++++++++ BIDS Data I/O ++++++++++++++++++++++++++ #
        # folder -> select files
        self.workflow.connect(
            self.subject_source, "subject_id",
            self.select_files, "subject_id",
        )
        # select files -> obtain bvec/bval path as tuple
        self.workflow.connect(
            self.select_files, "b_files",
            self.bfiles_input, "in_List",
        )
        
        # +++++++++++++++++++++++++++++ ACPC HCP Alignment +++++++++++++++++++++++++++++ #
        # selected anatomy files -> robustFOV (FSL) etc
        self.workflow.connect(
            [
                (self.select_files, self.reduceFOV, [("anat", "in_file")]),
                (self.reduceFOV, self.xfminverse, [("out_transform", "in_file")]),
                (self.reduceFOV, self.flirt, [("out_roi", "in_file")]),
                (self.xfminverse, self.concat_xfm, [("out_file", "in_filee")]),
                (self.flirt, self.concat_xfm, [("out_matrix_file", "in_file2")]),
                (self.concat_xfm, self.alignxfm, [("out_file", "in_file")]),
                (self.select_files, self.ACPC_warp, [("anat", "in_file")]),
                (self.alignxfm, self.ACPC_warp, [("out_file", "premat")]),
                (self.ACPC_warp, self.datasink, [("out_file", "acpc_aligned.@t1_acpc")])
            ]
        )
        # outputs of these nodes to datasink:
#         self.workflow.connect(
#             [(self.reduceFOV, self.datasink, [("out_transform", "acpc_aligned.@out_xfm")]),
#              (self.reduceFOV, self.datasink, [("out_roi", "acpc_aligned.@out_roi")]),
#              (self.xfminverse, self.datasink, [("out_file", "acpc_aligned.@full2roi")]),
#              (self.flirt, self.datasink, [("out_matrix_file", "acpc_aligned.@roi2std")]),
#              (self.flirt, self.datasink, [("out_file", "acpc_aligned.@acpc_mni")]),
#              (self.concat_xfm, self.datasink, [("out_file", "acpc_aligned.@full2std")]),
#              (self.alignxfm, self.datasink, [("out_file", "acpc_aligned.@outputmatrix")]),
#              (self.ACPC_warp, self.datasink, [("out_file", "acpc_aligned.@t1_acpc")])
#             ]
#         )
        
        # +++++++++++++++++++++++++++++ Freesurfer ReconAll +++++++++++++++++++++++++++++ #
        # ReconAll connection:
        self.workflow.connect(
            [
                (self.ACPC_warp, self.reconall, [("out_file", "T1_files")]),
                (self.reconall, self.datasink, [("T1", "reconall,@T1"),
                                               ("wm", "reconall.@wm"),
                                               ("wmparc", "reconall.@wmparc")])
            ]
        )
        
        # +++++++++++++++++++++++++++++ Preps, checks and conversions +++++++++++++++++++++++++++++ #
        # convert dwi to mrtrix3 image file format:
        self.workflow.connect(
            [
                (self.select_files, self.mrconvert, [("dwi", "in_file")]),
                (self.select_files, self.sub_grad_files, [("dwi", "sub_dwi")]),
                (self.sub_grad_files , self.mrconvert, [("fslgrad", "grad_fsl")]),
                (self.mrconvert, self.createMask, [("out_file", "in_file")]),
                (self.mrconvert, self.GradCheck, [("out_file", "in_file")]),
                (self.createMask, self.GradCheck, [("out_file", "mask_file")]),
                (self.mrconvert, self.GradCheck, [("out_bfile", "grad_file")]),
                (self.GradCheck, self.NewGradMR, [("out_bfile", "grad_file")]),
                (self.mrconvert, self.NewGradMR, [("out_file", "in_file")])
            ]
        )
        # prepmrtrix to datasink (are these files actually needed in datasink?):
#         self.workflow.connect(
#             [
#                 (self.NewGradMR, self.datasink, [("out_file", "prepmrtrix.@cdwi")]),
#                 (self.GradCheck, self.datasink, [("out_bfile", "prepmrtrix.@cgrad")]),
#                 (self.createMask, self.datasink, [("out_file", "prepmrtrix.@brainmask")]),
#                 (self.mrconvert, self.datasink, [("out_")])
#             ]
#         )
        
        # +++++++++++ Denoise/Gibbs/fslpreproc/dwibiascorrect/rician/intensity norm ++++++++++ #
        # selected dwi -> denoise
        self.workflow.connect(
            [
                (self.NewGradMR, self.denoise, [("out_file", "in_file")]),
                (self.denoise, self.datasink, [("noise", "preproc.@noise")]),
                (self.denoise, self.DeGibbs, [("out_file", "in_file")]),
                (self.DeGibbs, self.fslpreproc, [("out_file", "in_file")]),
                (self.GradCheck, self.fslpreproc, [("out_bfile", "grad_file")]),
                (self.fslpreproc, self.datasink, [("out_file", "preproc.@fslpreproc")]),
                (self.fslpreproc, self.biascorrect, [("out_file", "in_file")]),
                (self.GradCheck, self.biascorrect, [("out_bfile", "grad_file")]),
                (self.biascorrect, self.datasinnk, [("bias", "preproc.@biasfield")]),
                (self.biascorrect, self.grad_info, [("out_file", "in_file")]),
                (self.GradCheck, self.grad_info, [("out_bfile", "grad_file")]),
                (self.denoise, self.low_noise_map, [("noise", "isfinite")]),
                (self.denoise, self.low_noise_map, [("noise", "cond_if")]),
                (self.low_noise_map, self.rician_noise, [("out_file", "lownoisemap")]),
                (self.biascorrect, self.rician_noise, [("out_file", "in_file")]),
                (self.rician_noise, self.check_rician, [("out_file", "isfinite")]),
                (self.rician_noise, self.check_rician, [("out_file", "cond_if")]),
                (self.check_rician, self.convert_rician, [("out_file", "in_file")]),
                (self.grad_info, self.convert_rician, [("out_bfile", "grad_file")]),
                (self.convert_rician, self.datasink, [("out_file", "preproc.@RCN_corrected")]),
                (self.convert_rician, self.dwi_mask, [("out_file", "in_file")]),
                (self.dwi_mask, self.fit_tensor, [("out_file", "in_mask")]),
                (self.convert_rician, self.fit_tensor, [("out_file", "in_file")]),
                (self.fit_tensor, self.tensor_FA, [("out_file", "in_file")]),
                (self.tensor_FA, self.wm_mask, [("out_fa", "in_file")]),
                (self.wm_mask, self.norm_intensity, [("out_file", "mask_file")]),
                (self.converted_rician, self.norm_intensity, [("out_file", "in_file")]),
                (self.tensor_FA, self.datasink, [("out_fa", "preproc.@FA")]),
                (self.wm_mask, self.datasinnk, [("out_file", "preproc.@wm_mask")]),
                (self.norm_intensity, self.datasink, [("out_file", "preproc.@dwi_norm")])
            ]
        )
        
        # +++++++++++++++++++++++++++++ B0 Volume/Mask +++++++++++++++++++++++++++++ #
        self.workflow.connect(
            [
                (self.norm_intensity, self.sub_b0extract, [("out_file", "in_file")]),
                (self.sub_b0extract, self.sub_b0mean, [("out_file", "in_file")]),
                (self.norm_intensity, self.sub_b0mask, [("out_file", "in_file")]),
                (self.sub_b0mask, self.sub_convert_mask, [("out_file", "in_file")]),
                (self.sub_b0mean, self.sub_convert_dwi, [("out_file", "in_file")]),
                (self.sub_convert_dwi, self.sub_apply_mask, [("out_file", "in_file")]),
                (self.sub_convert_mask, self.sub_apply_mask, [("out_file", "mask_file")])
            ]
        )
                                                        
        # +++++++++++++++++++++++++++++ Align DWI to ACPC MNI +++++++++++++++++++++++++++++ #
        
       
         # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        # Align DWI to ANAT w/ ACPC HCP MNI152 space
        # first extract the brain:
        self.t1_bet = MapNode(fsl.preprocess.BET(), name='fsl_bet', iterfield="in_file")
        # align DWI to ANAT... get transformation matrix first
        self.epi_reg = MapNode(fsl.epi.EpiReg(), name='fsl_epireg', iterfield='epi')
        # convert transformation matrix to mrtrix format
        self.acpc_xfm = MapNode(ppt.TransConvert(), name='transformconvert', iterfield="flirt_xfm")
        # apply the transformation:
        self.apply_xfm = MapNode(ppt.MRTransform(), name='mrtransform', iterfield="in_file")
        
        # b0 extractions:
        self.workflow.connect(
            self.mrt_preproc, "out_file", self.b0extract, "in_file",
        )
        self.workflow.connect(
            self.bfiles_input, "out_path", self.b0extract, "grad_fsl",
        )
        self.workflow.connect(
            self.b0extract, "out_file", self.b0mean, "in_file",
        )
        self.workflow.connect(
            self.b0mean, "out_file", self.fsl_bet, "in_file",
        )
        # extractions --> datasink:
        self.workflow.connect(
            self.b0extract, "out_file", self.datasink, "extraction.@volume",
        )
        self.workflow.connect(
            self.b0mean, "out_file", self.datasink, "extraction.@mean",
        )
        self.workflow.connect(
            self.fsl_bet, "mask_file", self.datasink, "extraction.@BET_mask",
        )
        self.workflow.connect(
            self.fsl_bet, "out_file", self.datasink, "extraction.@BET_image",
        )
        # coregistration of atlases to DWI, select atlases as moving image
        self.workflow.connect(
            self.atlas_source, 'atlas_name', self.select_atlas, 'file_name'
        )
        self.workflow.connect(
            self.select_atlas, 'atlas', self.linear_coreg, 'moving_image'
        )
        # use extracted brain as target brain for co-registration, no dwi noise
        self.workflow.connect(
            self.fsl_bet, 'out_file', self.linear_coreg, 'fixed_image'
        )
        self.workflow.connect(
            self.fsl_bet, 'out_file', self.nonlinear_coreg, 'fixed_image'
        )
        self.workflow.connect(
            self.linear_coreg, 'warped_image', self.nonlinear_coreg, 'moving_image'
        )
        # coreg --> datasink
#         self.workflow.connect(
#             self.linear_coreg, 'warped_image', self.datasink, 'coreg.@affine_atlas'
#         )
#         self.workflow.connect(
#             self.nonlinear_coreg, 'warped_image', self.datasink, 'coreg.@syn_atlas'
#         )
        
        
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #    
    def default_setup(self, phase_encoding_design = "-rpe_none", phase_encoding_dir = "j-"):
        """
        Set up pipeline parameters with all default settings blindly
        You want to use this for a quick test run. Then tweak parameters after examining all output images
        Note that this assumes input options for eddy current correction as "--slm=linear" 
        Arguments:
            phase_encoding_design (str): defaults to "-rpe_none", or no reverse phase encoding volume, eddy current and motion correction only, other options include "rpe_pair, rpe_all, rpe_header"
            phase_encoding_dir (str): defaults to "j-", or anterior to posterior. (Options: i,j,k or RL, PA, IS)
        """
        self.denoise_inputs(force = True, quiet = True)
        self.DeGibbs_inputs()
        self.ants_bfc_inputs()
        self.mrt_preproc_inputs(
            rpe_options=phase_encoding_design,
            pe_dir=phase_encoding_dir,
            eddy_options='"--slm=linear --verbose"',
            nthreads=4,
            )
        self.b0extract_inputs()
        self.b0mean_inputs()
        self.bet_inputs()
        self.atlas_inputs(atlas_dir=str(input('Please indicate directory with atlas volumes: ')), atlas_names=list(input('Please indicate list of selected atlas names: ')))
        self.linear_coreg_inputs() #defaults
        self.nonlinear_coreg_inputs() #defaults
        self.connect_nodes(wf_name = 'default_workflow')
        return print(self.__dict__)
      
        
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
    def draw_pipeline(self):
        self.workflow.write_graph(graph2use='orig', dotfilename = 'pipetography.dot')
    
    
    # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
    def run_pipeline(self, parallel = True):
        if parallel is True:
            processes = int(input('Number of Processes: '))
            self.workflow.run('MultiProc', plugin_args = {'n_procs': processes})

Example:

In [ ]:
#create pipeline:
preproc_dwi = pipeline()

Creating layout of data directory, might take a while if there are a lot of subjects


Check if our environment is properly set-up:

In [ ]:
#example
preproc_dwi.check_environment()

FSLOUTPUTTYPE is valid
FSLDIR is valid
ANTS is valid
mrtrix3 is valid


In [ ]:
#example
#set output destination:
preproc_dwi.set_datasink()

Please indicate an output directory:  'output'


Take a look at what's in the `pipeline`:

In [ ]:
preproc_dwi.__dict__

{'data_dir': 'data',
 'sub_list': ['11048'],
 'layout': BIDS Layout: ...ers/xxie/lab/pipetography/data | Subjects: 1 | Sessions: 1 | Runs: 0,
 'dwi_file': 'sub-{subject_id}/ses-*/dwi/sub-{subject_id}_ses-*_dwi.nii.gz',
 'b_files': 'sub-{subject_id}/ses-1/dwi/sub-{subject_id}_ses-1_dwi.bv*',
 'sub_template': {'dwi': 'sub-{subject_id}/ses-*/dwi/sub-{subject_id}_ses-*_dwi.nii.gz',
  'b_files': 'sub-{subject_id}/ses-1/dwi/sub-{subject_id}_ses-1_dwi.bv*'},
 'sub_source': data_source,
 'select_files': select_files,
 'bfiles_input': select_bfiles,
 'denoise': denoise,
 'ringing': ringing_removal,
 'ants_bfc': ants_bias_correct,
 'mrt_preproc': mrtrix3_preproc,
 'atlas_dir': None,
 'atlas_names': None,
 'atlas_source': None,
 'select_atlas': None,
 'b0extract': dwiextract,
 'b0mean': mrmath,
 'fsl_bet': brain_extraction,
 'linear_coreg': linear_registration,
 'nonlinear_coreg': nonlinear_registration,
 'datasink': datasink,
 'workflow': None}

We can set up preprocessing pipeline with default parameters:

In [ ]:
#example
#usage
preproc_dwi.default_setup()

Please indicate directory with atlas volumes:  '/Users/xxie/lab/atlases'
Please indicate list of selected atlas names:  ['BN_Atlas_246_2mm.nii','DK_atlas86_1mm.nii']


{'data_dir': 'data', 'sub_list': ['11048'], 'layout': BIDS Layout: ...ers/xxie/lab/pipetography/data | Subjects: 1 | Sessions: 1 | Runs: 0, 'dwi_file': 'sub-{subject_id}/ses-*/dwi/sub-{subject_id}_ses-*_dwi.nii.gz', 'b_files': 'sub-{subject_id}/ses-1/dwi/sub-{subject_id}_ses-1_dwi.bv*', 'sub_template': {'dwi': 'sub-{subject_id}/ses-*/dwi/sub-{subject_id}_ses-*_dwi.nii.gz', 'b_files': 'sub-{subject_id}/ses-1/dwi/sub-{subject_id}_ses-1_dwi.bv*'}, 'sub_source': default_workflow.data_source, 'select_files': default_workflow.select_files, 'bfiles_input': default_workflow.select_bfiles, 'denoise': default_workflow.denoise, 'ringing': default_workflow.ringing_removal, 'ants_bfc': default_workflow.ants_bias_correct, 'mrt_preproc': default_workflow.mrtrix3_preproc, 'atlas_dir': "'/Users/xxie/lab/atlases'", 'atlas_names': ['[', "'", 'B', 'N', '_', 'A', 't', 'l', 'a', 's', '_', '2', '4', '6', '_', '2', 'm', 'm', '.', 'n', 'i', 'i', "'", ',', "'", 'D', 'K', '_', 'a', 't', 'l', 'a', 's', '8', '6', 

OR we can fill this in one by one:

First, let's tell the pipeline where we have atlas volumes and which ones to use:

In [ ]:
#example
preproc_dwi.atlas_inputs(atlas_dir = '/Users/xxie/lab/atlases', atlas_names = ['BN_Atlas_246_2mm.nii','DK_atlas86_1mm.nii'])

Next, we give inputs to the `denoise` Node:

In [ ]:
#example
preproc_dwi.denoise_inputs(force = True, quiet = True)
# we kept everything else default, like output names and number of threads.

Gibbs ringing removal:

In [ ]:
#example
preproc_dwi.DeGibbs_inputs() # keep it default
print('Output file name is ' + preproc_dwi.ringing.inputs.out_file)

ANTs Bias Field Correction:

In [ ]:
#example
preproc_dwi.ants_bfc_inputs() # keep it default:
preproc_dwi.ants_bfc.inputs.print_traits

`in_file` is undefined because we will feed it this information once we connect our individual functions.

Next let's set up eddy current/motion correction, we need to tell the pipeline the phase encoding settings and inputs to eddy algorithm:

In [ ]:
#example
preproc_dwi.mrt_preproc_inputs(
    rpe_options="-rpe_none",
    pe_dir="j-",
    eddy_options='"--slm=linear --verbose"',
    nthreads=4,
)
preproc_dwi.mrt_preproc.inputs.print_traits

`grad_fsl` and `in_file` will be input via connected workflow later

Next, we set-up brain extraction from B0 volumes:

In [ ]:
#example
# Extract b0 volumes:
preproc_dwi.b0extract_inputs() #default
# Create average B0 volume:
preproc_dwi.b0mean_inputs() #default
# Extract brain from B0 average volume:
preproc_dwi.bet_inputs() #defaults again, using FSL's BET

ANTs registration set up:

In [ ]:
#example
preproc_dwi.linear_coreg_inputs() #defaults
preproc_dwi.nonlinear_coreg_inputs() #defaults
preproc_dwi.nonlinear_coreg.inputs.print_traits

All the default settings may not be optimal for your dataset, run the processing for a test image and see the intermediate results and then tweak the available inputs to improve the output image quality.

Now that our pre-processing Nodes are properly setup, we can connect and create a workflow:

In [ ]:
#example
preproc_dwi.connect_nodes(wf_name = 'pipetography_workflow')
preproc_dwi.draw_pipeline()

Let's take a look at the final workflow we created:

In [ ]:
#example
from IPython.display import Image
Image('data/derivatives/test_run/pipetography_detailed.png')